In [324]:
import requests
import os
import pandas as pd
import dotenv
import datetime

In [325]:
#########################################  1ers essais #################################################
dotenv.load_dotenv()
api_key = os.getenv("API_KEY")

params = {
    "q" : "Lille,fr",
    "appid" : api_key
}

response = requests.get("https://api.openweathermap.org/data/2.5/weather", params=params)
response.url

'https://api.openweathermap.org/data/2.5/weather?q=Lille%2Cfr&appid=8b98ee62cd01073dfc092c21307e4714'

In [326]:
######################################### Essais avec 20 villes #################################################

api_key = os.getenv("API_KEY")
villes = ['Paris', 'Marseille', 'Lyon', 'Toulouse', 'Nice', 'Nantes', 'Strasbourg', 'Montpellier', 'Bordeaux', 'Lille', 'Rennes', 'Reims', 'Le Havre', 'Saint-Étienne', 'Toulon', 'Grenoble', 'Dijon', 'Angers', 'Nîmes', 'Villeurbanne']
resultat = []
for ville in villes:
    params = {"appid" : api_key}
    params["q"] = ville+",fr"
    resultat.append(eval(requests.get("https://api.openweathermap.org/data/2.5/weather", params=params).text))

In [327]:
df = pd.DataFrame(resultat)
df.head(3) # Les dictionnaires dans les dictionnaires seront difficiles a utiliser avec pandas

,coord,weather,base,main,visibility,wind,clouds,dt,sys,timezone,id,name,cod
0,"{'lon': 2.3488, 'lat': 48.8534}","[{'id': 800, 'main': 'Clear', 'description': '...",stations,"{'temp': 284.83, 'feels_like': 283.36, 'temp_m...",10000,"{'speed': 7.72, 'deg': 60}",{'all': 0},1680530311,"{'type': 2, 'id': 2041230, 'country': 'FR', 's...",7200,2988507,Paris,200
1,"{'lon': 5.5, 'lat': 43.3333}","[{'id': 800, 'main': 'Clear', 'description': '...",stations,"{'temp': 291.3, 'feels_like': 290.08, 'temp_mi...",10000,"{'speed': 4.47, 'deg': 272, 'gust': 9.39}",{'all': 0},1680530289,"{'type': 2, 'id': 80238, 'country': 'FR', 'sun...",7200,2995468,Arrondissement de Marseille,200
2,"{'lon': 4.5833, 'lat': 45.75}","[{'id': 803, 'main': 'Clouds', 'description': ...",stations,"{'temp': 280.77, 'feels_like': 279.77, 'temp_m...",10000,"{'speed': 1.79, 'deg': 63, 'gust': 4.02}",{'all': 67},1680530936,"{'type': 2, 'id': 2007821, 'country': 'FR', 's...",7200,2996943,Arrondissement de Lyon,200


In [328]:
# je les applatis donc grace a un focntion recursive qui parcourt chaque niveau de dictionnaire
def flatten_dict(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

for d in resultat:
    d["weather"] = d["weather"][0]

flat_results = []
for data_dict in resultat:
    flat_results.append(flatten_dict(data_dict))

In [329]:
df = pd.DataFrame(flat_results)
df.head()

,coord_lon,coord_lat,weather_id,weather_main,weather_description,weather_icon,base,main_temp,main_feels_like,main_temp_min,...,sys_country,sys_sunrise,sys_sunset,timezone,id,name,cod,wind_gust,main_sea_level,main_grnd_level
0,2.3488,48.8534,800,Clear,clear sky,01d,stations,284.83,283.36,283.77,...,FR,1680499491,1680546168,7200,2988507,Paris,200,NaN,NaN,NaN
1,5.5000,43.3333,800,Clear,clear sky,01d,stations,291.30,290.08,287.68,...,FR,1680499018,1680545129,7200,2995468,Arrondissement de Marseille,200,9.39,NaN,NaN
2,4.5833,45.7500,803,Clouds,broken clouds,04d,stations,280.77,279.77,280.18,...,FR,1680499122,1680545465,7200,2996943,Arrondissement de Lyon,200,4.02,NaN,NaN
3,1.4437,43.6043,804,Clouds,overcast clouds,04d,stations,286.16,285.14,285.80,...,FR,1680499978,1680546116,7200,2972315,Toulouse,200,NaN,NaN,NaN
4,7.2500,44.0000,804,Clouds,overcast clouds,04d,stations,282.88,281.51,282.05,...,FR,1680498567,1680544739,7200,2990439,Arrondissement de Nice,200,1.72,1010.0,861.0


In [330]:
df = df[["name", "main_temp", "main_feels_like", "main_temp_min", "main_temp_max", "main_pressure", "main_humidity", "wind_speed", "wind_deg", "sys_sunrise", "sys_sunset"]]
df.head()
# Le résultat est déjà + parlant

,name,main_temp,main_feels_like,main_temp_min,main_temp_max,main_pressure,main_humidity,wind_speed,wind_deg,sys_sunrise,sys_sunset
0,Paris,284.83,283.36,283.77,286.22,1025,50,7.72,60,1680499491,1680546168
1,Arrondissement de Marseille,291.30,290.08,287.68,292.41,1009,35,4.47,272,1680499018,1680545129
2,Arrondissement de Lyon,280.77,279.77,280.18,281.32,1019,74,1.79,63,1680499122,1680545465
3,Toulouse,286.16,285.14,285.80,286.38,1019,62,5.66,320,1680499978,1680546116
4,Arrondissement de Nice,282.88,281.51,282.05,283.04,1010,58,2.72,211,1680498567,1680544739


In [331]:
# Il me reste maintenant a transformer certaines données pour qu'elles soient + compréhensibles
# ex : la temp de Kelvin à °C / l'heure du lever/coucher du soleil en hh:mm:ss

df["main_temp"] = df["main_temp"] - 273.15
df["main_feels_like"] = df["main_feels_like"] - 273.15
df["main_temp_min"] = df["main_temp_min"] - 273.15
df["main_temp_max"] = df["main_temp_max"] - 273.15


df["sys_sunrise"] = df["sys_sunrise"].apply(lambda x: datetime.datetime.fromtimestamp(x))
df["sunrise_hour"] = df["sys_sunrise"].apply(lambda x: x.strftime('%H:%M'))
df["sys_sunset"] = df["sys_sunset"].apply(lambda x: datetime.datetime.fromtimestamp(x))
df["sunset_hour"] = df["sys_sunset"].apply(lambda x: x.strftime('%H:%M'))

df = df.drop(columns=["sys_sunrise", "sys_sunset"], axis=1)
df.head()

,name,main_temp,main_feels_like,main_temp_min,main_temp_max,main_pressure,main_humidity,wind_speed,wind_deg,sunrise_hour,sunset_hour
0,Paris,11.68,10.21,10.62,13.07,1025,50,7.72,60,07:24,20:22
1,Arrondissement de Marseille,18.15,16.93,14.53,19.26,1009,35,4.47,272,07:16,20:05
2,Arrondissement de Lyon,7.62,6.62,7.03,8.17,1019,74,1.79,63,07:18,20:11
3,Toulouse,13.01,11.99,12.65,13.23,1019,62,5.66,320,07:32,20:21
4,Arrondissement de Nice,9.73,8.36,8.90,9.89,1010,58,2.72,211,07:09,19:58


In [332]:
df.to_csv("weather_data_fr_20.csv")